In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
import csv
import os

In [363]:
def back_to_dict(dict_filename,df_filename):
    df=pd.read_csv(df_filename,index_col=[0],sep=';|,',engine='python',parse_dates=[0],infer_datetime_format=True)
    df.index=df.index.tz_localize('UTC').tz_convert('Europe/Brussels')
    df_dict=pd.read_csv(dict_filename,header=None)
    new_dict={'df':df}
    for i in df_dict.index:
        if i not in [4,5,6,8,10,11]:
            try:
                new_dict.update({str(df_dict.iloc[i,0]):float(df_dict.iloc[i,1])})
            except:
                new_dict.update({str(df_dict.iloc[i,0]):(df_dict.iloc[i,1])})
        else:
            try:
                a=np.array([float(i) for i in df_dict.iloc[i,1].replace('[','').replace(']','').split()])
            except:
                a=np.array([float(i) for i in df_dict.iloc[10,1].replace('[','').replace(']','').split(',')])
            finally:
                new_dict.update({str(df_dict.iloc[i,0]):a})
    return new_dict
            
    

In [364]:


def get_table_inputs():
    dwellings_US=np.array(['79', '466', '1', '539', '161', '122', '325', '327',
                    '538','403', '204', '164'])
    dwellings_CH=np.array(['110141955911' ,'1127347129716', '110698529950',
                       '1127343129682', '110699130179', '110696328886',
                       '110145356508', '110144156211', '110697229401',
                       '110697029296', '110698730039','110144756389'])
    country=np.hstack((np.tile('CH',12),np.tile('US',12)))
    cluster=np.tile(np.arange(0,12),2)
    clusters={'hh':np.hstack((dwellings_CH,dwellings_US)),'country':country,
              'cluster':cluster}
    clusters=pd.DataFrame(clusters)
    PV=[{'PV':3.2,'quartile':25,'country':'US'},
        {'PV':5,'quartile':50,'country':'US'},
        {'PV':6.4,'quartile':75,'country':'US'},
        {'PV':3.2,'quartile':25,'country':'CH'},
        {'PV':4.8,'quartile':50,'country':'CH'},
        {'PV':6.9,'quartile':75,'country':'CH'}]
    PV=pd.DataFrame(PV)
    App_comb_scenarios=np.array([i for i in itertools.product([False, True],repeat=3)])
    App_comb_scenarios=np.insert(App_comb_scenarios,slice(1,2),True,axis=1)
    App_comb=pd.DataFrame(App_comb_scenarios)
    App_comb=App_comb[0].map(str)+' '+App_comb[1].map(str)+' '+App_comb[2].map(
                    str)+' '+App_comb[3].map(str)
    App_comb=App_comb.reset_index()
    App_comb=App_comb.rename(columns={'index':'App_index',0:'App_comb'})
    conf=pd.DataFrame(np.array([[False,True,False,False],[False,True,True,False],[False,True,True,True],[True,True,False,False],[True,True,True,False],[True,True,True,True]]))
    conf=conf[0].map(str)+' '+conf[1].map(str)+' '+conf[2].map(
                str)+' '+conf[3].map(str)
    conf=conf.reset_index()
    conf=conf.rename(columns={'index':'conf_index',0:'conf'})
    house_types=pd.DataFrame(np.array(['SFH100','SFH45','SFH15']),columns=['House_type'])
    hp_types=pd.DataFrame(np.array(['GSHP','ASHP']),columns=['HP_type'])
    rad_types=pd.DataFrame(np.array(['under','rad']),columns=['Rad_type'])
    clusters.hh=clusters.hh.astype(float)

    return[clusters,PV,App_comb,conf,house_types,hp_types,rad_types]


In [390]:
df_filename=Path('../Output/df_110144756389_CH_PV50_NMC_1111_7.0_0100_mean.csv')
dict_filename=Path('../Output/dict_110144756389_CH_PV50_NMC_1111_7.0_0100_mean.csv')
dict_res=back_to_dict(dict_filename,df_filename)

In [391]:
[clusters,PV,App,conf,house_types,hp_types,rad_types]=get_table_inputs()

In [393]:
print('Main results')
df=dict_res['df']
App=App.App_index[App.App_comb==str(dict_res['App_comb']).replace(',','').strip(' ').replace('[','').replace(']','')].values
conf=conf.conf_index[conf.conf==str(dict_res['conf']).replace(',','').strip(' ').replace('[','').replace(']','')].values


df=df.apply(pd.to_numeric, errors='ignore')
agg_results=df.sum()
agg_results['App_comb']=App[0]
agg_results['conf']=conf[0]
agg_results=agg_results.drop(['SOC','Inv_P','Conv_P','price',
                              'Export_price'])
agg_results['SOC_mean']=df['SOC'].mean()/dict_res['Capacity']*100
agg_results['SOC_max']=df['SOC'].max()/dict_res['Capacity']*100
agg_results['SOC_min']=df['SOC'].min()/dict_res['Capacity']*100
agg_results['DoD_mean']=dict_res['DoD'].mean()*100
agg_results['DoD_max']=dict_res['DoD'].max()*100
agg_results['DoD_min']=dict_res['DoD'].min()*100
agg_results['DoD_min']=dict_res['DoD'].min()*100
agg_results['last_cap']=dict_res['Cap_arr'][-1]
agg_results['cap_fading']=(1-dict_res['Cap_arr'][-1]/
           dict_res['Capacity'])*100

agg_results['last_SOH']=dict_res['SOH'][-1]
agg_results['P_max_year_batt']=dict_res['P_max'].max()
agg_results['P_max_year_nbatt']=df['E_demand'].max()*4
agg_results['Capacity']=dict_res['Capacity']
agg_results['Tech']=dict_res['Tech']
agg_results['PV_nom']=dict_res['PV_nom']
agg_results['cluster']=clusters[clusters.hh==int(
        dict_res['name'].split('_')[0])].cluster.values
agg_results['country']=dict_res['name'].split('_')[1]
agg_results['quartile']=PV[(PV.PV==dict_res['PV_nom'])&
           (PV.country==dict_res['name'].split('_')[1])].quartile.values[0]

Main results


In [380]:
agg_results.P_max_year_batt

6.31358239

In [379]:
agg_results.P_max_year_nbatt

4.04

In [384]:
PV_cols = [col for col in df.columns if 'PV' in col]


In [385]:
PV_cols

['E_PV_batt',
 'E_PV_bu',
 'E_PV_bu2',
 'E_PV_curt',
 'E_PV_grid',
 'E_PV_hp',
 'E_PV_hp2',
 'E_PV_load',
 'E_loss_inv_PV',
 'E_PV']

In [403]:
def get_technical_indicators(App,agg_results,country,df,conf):
    #TSC
    agg_results['TSC']=(agg_results.E_PV_load+agg_results.E_PV_batt+agg_results.E_PV_bu+agg_results.E_PV_bu2+agg_results.E_PV_hp+agg_results.E_PV_hp2)/agg_results.E_PV*100#[%]
    agg_results['DSC']=(agg_results.E_PV_load+agg_results.E_PV_bu+agg_results.E_PV_bu2+agg_results.E_PV_hp+agg_results.E_PV_hp2)/agg_results.E_PV*100#[%]
    agg_results['ISC']=(agg_results.E_PV_batt)/agg_results.E_PV*100#[%]
    agg_results['EFC_nolifetime']=(agg_results.E_dis)/dict_res['Capacity']
    print(agg_results.TSC)
    print(agg_results.DSC)
    print(agg_results.ISC)
    print(agg_results.EFC_nolifetime)

In [404]:
get_technical_indicators(App,agg_results,'CH',df,3)

50.67785547929873
50.67785547929873
0.0
0.0


In [405]:
agg_results.E_demand

4433.860000000327

In [408]:
[col for col in df.columns if 'hp' in col]


['Bool_hp',
 'Bool_hp2',
 'E_PV_hp',
 'E_PV_hp2',
 'E_batt_hp',
 'E_batt_hp2',
 'E_grid_hp',
 'E_grid_hp2',
 'E_hp',
 'E_hp2',
 'E_hp_cooling',
 'Q_hp_dhwst',
 'Q_hp_sh']

In [409]:
agg_results.E_hp

8217.971573496889

In [410]:
agg_results.E_hp2

0.0